In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime

In [2]:
# set current time and date
def options_arbitrage(ticker, expiration_date, print_tables=False, to_latex=False):
    # set time
    now = datetime.now()
    
    # select the stock
    stock = yf.Ticker(ticker)

    # get latest market price
    price = stock.info['currentPrice']

    # get the option chain for the expiration date
    opt = stock.option_chain(expiration_date)
    
    # get the calls and puts
    calls = opt.calls
    puts = opt.puts

    calls = calls[['contractSymbol','lastTradeDate','strike', 'lastPrice', 'bid', 'ask', 'volume']].rename(columns={'strike': 'strike', 'lastPrice': 'call_lastPrice', 'bid': 'call_bid', 'ask': 'call_ask', 'volume': 'call_volume', 'lastTradeDate': 'call_lastTradeDate', 'contractSymbol': 'call_contractSymbol'})
    puts = puts[['contractSymbol','lastTradeDate','strike', 'lastPrice', 'bid', 'ask', 'volume']].rename(columns={'strike': 'strike', 'lastPrice': 'put_lastPrice', 'bid': 'put_bid', 'ask': 'put_ask', 'volume': 'put_volume', 'lastTradeDate': 'put_lastTradeDate', 'contractSymbol': 'put_contractSymbol'})

    merge = pd.merge(left=calls, right=puts, left_on='strike', right_on='strike', how='inner')

    # check length of the merge has at least 2 rows
    if len(merge) < 2:
        return ValueError('Not enough data to calculate put-call parity for 2 strike prices')

    strike1 = merge[merge['strike'] < price].iloc[-1]['strike']
    strike2 = merge[merge['strike'] > price].iloc[0]['strike']


    # filter merge, calls and puts to only include the relevant strike prices
    relevant_merge = merge[(merge['strike'] == strike1) | (merge['strike'] == strike2)].reset_index(drop=True)
    relevant_calls = calls[(calls['strike'] == strike1) | (calls['strike'] == strike2)].reset_index(drop=True)
    relevant_puts = puts[(puts['strike'] == strike1) | (puts['strike'] == strike2)].reset_index(drop=True)

    relevant_merge['stock_price'] = price

    # calculate the put-call parity
    relevant_merge['parity'] = relevant_merge['call_bid'] + relevant_merge['strike'] - (relevant_merge['put_bid'] + relevant_merge['stock_price'])

    relevant_merge['dowload_time'] = now 

    # reorder the columns
    merge_latex = relevant_merge[['call_bid', 'strike', 'put_bid', 'stock_price', 'parity', 'dowload_time']]
    merge_latex = merge_latex.rename(columns={'call_bid': '$C_0$', 'strike': '$X$', 'put_bid': '$P_0$', 'stock_price': '$S_0$', 'parity': 'Deviation', 'dowload_time': 'Download Time'})

    if print_tables:
        print(f'Call Options for {ticker} with expiration date {expiration_date}')
        display(relevant_calls)
        print(f'Put Options for {ticker} with expiration date {expiration_date}')
        display(relevant_puts)
        print(f'Put-Call Parity for {ticker} with expiration date {expiration_date}')
        display(merge_latex)
    
    if to_latex:        

        latex_call = relevant_calls.to_latex(index=False, float_format='%.2f', caption=f'Call Options for {ticker} with expiration date {expiration_date}')
        latex_put = relevant_puts.to_latex(index=False, float_format='%.2f', caption=f'Put Options for {ticker} with expiration date {expiration_date}')
        latex_merge = merge_latex.to_latex(index=False, float_format='%.2f', caption=f'Put-Call Parity for {ticker} with expiration date {expiration_date}')

        # replace toprule, midrule, bottomrule with hline
        latex_call = latex_call.replace('toprule', 'hline').replace('midrule', 'hline').replace('bottomrule', 'hline')
        latex_put = latex_put.replace('toprule', 'hline').replace('midrule', 'hline').replace('bottomrule', 'hline')
        latex_merge = latex_merge.replace('toprule', 'hline').replace('midrule', 'hline').replace('bottomrule', 'hline')

        latex_call = latex_call.replace('call_','').replace('begin{table}', 'begin{table}[H]').replace('+00:00','')
        latex_put = latex_put.replace('put_','').replace('begin{table}', 'begin{table}[H]').replace('+00:00','')
        latex_merge = latex_merge.replace('begin{table}', 'begin{table}[H]')

        print(latex_call)
        print(latex_put)
        print(latex_merge)

In [3]:
options_arbitrage(ticker='AMZN', expiration_date='2024-03-08', print_tables=True, to_latex=True)

Call Options for AMZN with expiration date 2024-03-08


,call_contractSymbol,call_lastTradeDate,strike,call_lastPrice,call_bid,call_ask,call_volume
0,AMZN240308C00165000,2024-02-08 20:33:22+00:00,165.0,8.0,0.0,0.0,482.0
1,AMZN240308C00170000,2024-02-08 20:54:58+00:00,170.0,4.7,0.0,0.0,247.0


Put Options for AMZN with expiration date 2024-03-08


,put_contractSymbol,put_lastTradeDate,strike,put_lastPrice,put_bid,put_ask,put_volume
0,AMZN240308P00165000,2024-02-08 20:55:40+00:00,165.0,2.18,0.0,0.0,305.0
1,AMZN240308P00170000,2024-02-08 20:59:32+00:00,170.0,4.15,0.0,0.0,263.0


Put-Call Parity for AMZN with expiration date 2024-03-08


,$C_0$,$X$,$P_0$,$S_0$,Deviation,Download Time
0,0.0,165.0,0.0,169.84,-4.84,2024-02-08 21:48:12.766203
1,0.0,170.0,0.0,169.84,0.16,2024-02-08 21:48:12.766203


\begin{table}[H]
\caption{Call Options for AMZN with expiration date 2024-03-08}
\begin{tabular}{llrrrrr}
\hline
contractSymbol & lastTradeDate & strike & lastPrice & bid & ask & volume \\
\hline
AMZN240308C00165000 & 2024-02-08 20:33:22 & 165.00 & 8.00 & 0.00 & 0.00 & 482.00 \\
AMZN240308C00170000 & 2024-02-08 20:54:58 & 170.00 & 4.70 & 0.00 & 0.00 & 247.00 \\
\hline
\end{tabular}
\end{table}

\begin{table}[H]
\caption{Put Options for AMZN with expiration date 2024-03-08}
\begin{tabular}{llrrrrr}
\hline
contractSymbol & lastTradeDate & strike & lastPrice & bid & ask & volume \\
\hline
AMZN240308P00165000 & 2024-02-08 20:55:40 & 165.00 & 2.18 & 0.00 & 0.00 & 305.00 \\
AMZN240308P00170000 & 2024-02-08 20:59:32 & 170.00 & 4.15 & 0.00 & 0.00 & 263.00 \\
\hline
\end{tabular}
\end{table}

\begin{table}[H]
\caption{Put-Call Parity for AMZN with expiration date 2024-03-08}
\begin{tabular}{rrrrrl}
\hline
$C_0$ & $X$ & $P_0$ & $S_0$ & Deviation & Download Time \\
\hline
0.00 & 165.00 & 0.00 & 1